In [27]:
import requests
import bs4
import pandas as pd

df_university = pd.read_csv('National_University_Ranking.csv')
base_url = 'https://en.wikipedia.org/wiki/'

In [31]:
num_of_fail = 0
failed_unversities = []
data_universities = []
motto_list_unversity = []

df_university = df_university.dropna(how='all')
i = 0
for string in df_university['Name/Rank']:
    motto = ''
    school_name = string.split('\n')[0]
    school_name_str = school_name.replace(' ', '_')
    response = requests.get(base_url + school_name_str)
    html = bs4.BeautifulSoup(response.text, "lxml")
   
    i += 1
    print(i)
    print(school_name)
    
    # If failed
    if 'Motto' not in html.get_text():
        # Launch a search for the unversity
        new_url_head = 'https://en.wikipedia.org/w/index.php?search='
        new_url_tail = '&title=Special:Search&fulltext=1'
        str_lst = school_name.split(' ')
        for st in str_lst:
            new_url_head += st
        
        response = requests.get(new_url_head + new_url_tail)
        html = bs4.BeautifulSoup(response.text, "lxml")
        if html.find_all(class_ = 'mw-search-results'):
            for child in html.find_all(class_ = 'mw-search-results')[0].children:
                new_url = base_url + child.div.a['href'][6:]
                break
        else:
            print('Failed to find motto for: ' + school_name_str)
            failed_unversities.append(school_name)
            data_universities.append([school_name, ' '])
            continue
        response = requests.get(new_url)
        html = bs4.BeautifulSoup(response.text, "lxml")
    
    # If the motto is not found again
    if 'Motto' not in html.get_text():
        print('Failed to find motto for: ' + school_name_str)
        failed_unversities.append(school_name)
        data_universities.append([school_name, ' '])
        continue
        
    # If didn't fail
    for child in html.tbody.children:
        if 'Motto in' in html.get_text():
            if child.th and child.th.div:
                if 'Motto in' in child.th.div.text and child.td.text:
                    motto = child.td.text
                    data_universities.append([school_name, motto])
                    motto_list_unversity.append(motto)
                    print(motto)
                    
        else:
            if child.th and 'Motto' in child.th.text:
                if child.td and child.td.text:
                    motto = child.td.text
                    data_universities.append([school_name, motto])
                    motto_list_unversity.append(motto)
                    print(motto)
                
    if motto == '':
        print('Failed to find motto for: ' + school_name_str)
        failed_unversities.append(school_name)
        data_universities.append([school_name, ' ']) 
                
df_university = pd.DataFrame(data_universities ,columns=['University','Moto'],dtype=str)
df_university.to_csv('University_mottos.csv')

1
Princeton University
Under God's Power She Flourishes[1]
2
Harvard University
Truth
3
Columbia University
In Thy light shall we see light[3] (Psalms 36:9)
4
Massachusetts Institute of Technology
Mind and Hand[1]
5
University of Chicago
Let knowledge grow from more to more; and so be human life enriched[1]
6
Yale University
Light and truth
7
Stanford University
The wind of freedom blows[1]
8
Duke University
Knowledge and Faith[2]
9
University of Pennsylvania
Laws without morals are useless
10
Johns Hopkins University
The Truth Shall Set You Free
11
Northwestern University
"Whatsoever things are true" (Philippians 4:8 AV)"The word full of grace and truth" (Gospel of John 1:14)
12
California Institute of Technology
The truth shall make you free[1]
13
Dartmouth College
A voice crying out in the wilderness[1]
14
Brown University
In God We Hope[1]
15
Vanderbilt University
Failed to find motto for: Vanderbilt_University
16
Cornell University
Failed to find motto for: Cornell_University
17
R

In [33]:
words = {}
special_char = ['[', ']', ':', '(', ')', '"', '.', ',', '/', '1', '2', '3']
for indx in range(len(motto_list_unversity)):
    tmp_string = motto_list_unversity[indx]
    for char in special_char:
        if char in tmp_string:
            tmp_string = tmp_string.replace(char, '')
    for word in tmp_string.split(' '):
        word = word.lower()
        if word in words:
            words[word] += 1
        else:
            words[word] = 1

In [34]:
words

{'': 2,
 "'for": 1,
 "'knowledge": 1,
 '4': 1,
 '48': 1,
 '69': 1,
 'a': 4,
 'accomplish': 1,
 'advancing': 1,
 'adversity': 1,
 'all': 1,
 'also': 1,
 'always': 1,
 'and': 23,
 'andrew': 1,
 'are': 2,
 'art': 2,
 'arts': 1,
 'avthe': 1,
 'be': 8,
 'bear': 1,
 'being': 1,
 'better': 3,
 'beyond': 1,
 'blows': 1,
 'bond': 1,
 'both': 1,
 'breadth': 1,
 'but': 2,
 'by': 1,
 'campus': 1,
 'carnegie': 1,
 'change': 1,
 'character': 1,
 'charity': 1,
 'church': 1,
 'citizensa': 1,
 'citizenship': 1,
 'city': 1,
 'common': 1,
 'conspicuous': 1,
 'country': 1,
 'crowns': 1,
 'crying': 1,
 'depends': 1,
 'depth': 1,
 'divine': 1,
 'do': 1,
 'earns': 1,
 'education': 3,
 'enriched': 1,
 'et': 1,
 'even': 1,
 'ever': 2,
 'excel': 2,
 'faith': 1,
 'flourishes': 1,
 'for': 11,
 'forth': 2,
 'free': 3,
 'freedom': 1,
 'freely': 2,
 'from': 3,
 'full': 1,
 'future': 1,
 'give': 1,
 'glory': 3,
 'go': 1,
 'god': 8,
 "god's": 1,
 'gospel': 1,
 'grace': 1,
 'great': 1,
 'greater': 2,
 'grow': 1,
 'hand

In [37]:
df_lac = pd.read_csv('LAC_Ranking.csv')

df_lac = df_lac.dropna(how='all')
data_lacs = []

num_of_fail_lac = 0
failed_lac = []
i = 0
for string in df_lac['Name/Rank']:
    motto = ''
    school_name = string.split('\n')[0].replace(' ', '_')
    response = requests.get(base_url + school_name)
    html = bs4.BeautifulSoup(response.text, "lxml")
    
    i += 1
    print(i)
    print(school_name)
    
    # If failed
    if 'Motto' not in html.get_text():
        new_url_head = 'https://en.wikipedia.org/w/index.php?search='
        new_url_tail = '&title=Special:Search&fulltext=1'
        str_lst = string.split('\n')[0].split(' ')
        for st in str_lst:
            new_url_head += st
        
        response = requests.get(new_url_head + new_url_tail)
        html = bs4.BeautifulSoup(response.text, "lxml")
        if html.find_all(class_ = 'mw-search-results'):
            for child in html.find_all(class_ = 'mw-search-results')[0].children:
                new_url = base_url + child.div.a['href'][6:]
                break
        else:
            print('Failed to find motto for: ' + school_name)
            failed_lac.append(string.split('\n')[0])
            continue
        response = requests.get(new_url)
        html = bs4.BeautifulSoup(response.text, "lxml")
        
    if 'Motto' not in html.get_text():
        print('Failed to find motto for: ' + school_name)
        failed_lac.append(string.split('\n')[0])
     
    
    # If didn't fail
    for child in html.tbody.children:
        if 'Motto in' in html.get_text():
            if child.th and child.th.div:
                if 'Motto in' in child.th.div.text and child.td.text:
                    motto = child.td.text
                    data_lacs.append([school_name, motto])
#                     motto_list_unversity.append(motto)
                    print(motto)
                    
        else:
            if child.th and 'Motto' in child.th.text:
                if child.td and child.td.text:
                    motto = child.td.text
                    data_lacs.append([school_name, motto])
#                     motto_list_unversity.append(motto)
                    print(motto)
                
    if motto == '':
        print('Failed to find motto for: ' + school_name_str)
        failed_lac.append(school_name)
        data_lacs.append([school_name, ' ']) 
        
#     # If didn't fail
#     for child in html.tbody.children:
#         if child.th and child.th.div and 'Motto' in child.th.div.text:
#             if child.td.text:
#                 motto = child.td.text
#                 data_lacs.append([school_name, motto])
#                 print('The motto of ' + string.split('\n')[0] + ' is "' + motto + '"')
                
df_lacs = pd.DataFrame(data_lacs, columns=['LAC','Moto'],dtype=str)
df_lacs.to_csv('LAC_mottos.csv')

1
Williams_College
"Through the Generosity of E. Williams, Esquire"[1]
2
Amherst_College
Let them enlighten the lands[1]
3
Swarthmore_College
Mind the Light[1][2]
4
Wellesley_College
Not to be ministered unto, but to minister
5
Bowdoin_College
As an eagle towards the sky
6
Carleton_College
The Revelation / Announcement of Your Words Illuminates
7
Middlebury_College
Knowledge and Virtue
8
Pomona_College
Failed to find motto for: Pomona_College
Failed to find motto for: University_of_Oregon
9
Claremont_McKenna_College
Civilization prospers with commerce[1]
10
Davidson_College
Let Learning Be Cherished Where Liberty Has Arisen
11
Grinnell_College
Truth and Humanity
12
Haverford_College
Failed to find motto for: University_of_Oregon
13
Smith_College
Failed to find motto for: University_of_Oregon
14
Vassar_College
Failed to find motto for: Vassar_College
Failed to find motto for: University_of_Oregon
15
Washington_and_Lee_University
"Not Unmindful of the Future"
16
Colgate_University
For Go

In [38]:
df_combined = pd.read_csv('combined.csv')

In [56]:
words = {}
special_char = ['[', ']', ':', '(', ')', '"', '.', ',', '/', '1', '2', '3']
for mott in df_combined['Moto']:
    for char in special_char:
        if char in mott:
            mott = mott.replace(char, '')
    
    for word in mott.split(' '):
        word = word.lower()
        if word in words:
            words[word] += 1
        else:
            words[word] = 1

words_out = {'Count': [], 'Word': []}
for word in words:
    words_out['Word'].append(word)
    words_out['Count'].append(words[word])

In [63]:
df_words = pd.DataFrame(words_out)
df_words = df_words.drop([12])
df_words.sort_values(by='Count', ascending=False).to_csv('Words_Count.csv')